In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from os.path import abspath

# SparkSession
URL_SPARK = "spark://spark-master:7077"
warehouse_location = abspath('spark-warehouse')

spark = (
    SparkSession.builder
    .appName("write-stream-elab9070-minio")
    .config("spark.executor.memory", "4g")
    .config("spark.executor.cores", "2")
    .config("spark.cores.max", "2")
    .config("spark.sql.warehouse.dir", warehouse_location)
    .config("spark.jars", "jars/hadoop-aws-3.3.2.jar,jars/aws-java-sdk-bundle-1.12.172.jar,jars/hadoop-common-3.3.2.jar,jars/spark-avro_2.12-3.3.2.jar,jars/commons-pool2-2.11.1.jar,jars/spark-sql-kafka-0-10_2.12-3.3.2.jar,jars/kafka-clients-2.8.1.jar,jars/spark-streaming-kafka-0-10-assembly_2.12-3.2.1.jar")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .master(URL_SPARK)
    .getOrCreate()
)


/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


23/10/22 21:03:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# import os
# os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
# os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://sparkjupyter-minio-1:9000"

# spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://sparkjupyter-minio-1:9000")
# spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minioadmin")
# spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minioadmin" )
# spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
# spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
# spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled","false")
import os
os.environ["AWS_ACCESS_KEY_ID"] = "minio_access_key"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio_secret_key"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://minio:9000"

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minio_access_key")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minio_secret_key" )
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled","false")

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime
import pyspark.sql.functions as F

In [4]:
df = spark \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "10.133.100.199:9092") \
            .option("subscribe", "telegraf1") \
            .option("startingOffsets", "earliest") \
            .load()
stringDF = df.selectExpr("CAST(timestamp AS STRING)","CAST(value AS STRING)")


In [5]:
# withColumn('timestamp', regexp_extract('timestamp', r'last_data_time=([^\Wi]+)', 1))\
df_generic_counter = stringDF.withColumn('source', regexp_extract('value', r'source=(.*?),', 1))\
        .withColumn('interface_name', regexp_extract('value', r'interface_name=(.*?),', 1))\
        .withColumn('packets_received', regexp_extract('value', r'packets_received=([^\Wi,bytes_received]+)', 1))\
        .withColumn('packets_sent', regexp_extract('value', r'packets_sent=([^\Wi]+)', 1)) \
        .withColumn('bytes_received', regexp_extract('value', r'bytes_received=([^\Wi]+)', 1))\
        .withColumn('bytes_sent', regexp_extract('value', r'bytes_sent=([^\Wi]+)', 1)) \
        .withColumn('multicast_packets_received', regexp_extract('value', r'multicast_packets_received=([^\Wi]+)', 1))\
        .withColumn('multicast_packets_sent',regexp_extract('value', r'multicast_packets_sent=([^\Wi]+)', 1))\
        .withColumn('broadcast_packets_received', regexp_extract('value', r'broadcast_packets_received=([^\Wi]+)', 1))\
        .withColumn('broadcast_packets_sent',regexp_extract('value', r'broadcast_packets_sent=([^\Wi]+)', 1))\
        .withColumn('output_drops',regexp_extract('value', r'output_drops=([^\Wi]+)', 1))\
        .withColumn('output_queue_drops',regexp_extract('value', r'output_queue_drops=([^\Wi]+)', 1))\
        .withColumn('input_drops',regexp_extract('value', r'input_drops=([^\Wi]+)', 1))\
        .withColumn('input_queue_drops',regexp_extract('value', r'input_queue_drops=([^\Wi]+)', 1))\
        .withColumn('input_errors',regexp_extract('value', r'input_errors=([^\Wi]+)', 1))\
        

In [6]:
df_generic_counter = df_generic_counter.drop('value')
df_generic_counter = df_generic_counter.filter(df_generic_counter.interface_name!='')

In [7]:
# df_generic_counter = df_generic_counter.withColumn('timestampType',F.from_unixtime(F.col('timestamp')))

In [8]:
# df_generic_counter.writeStream.format('console').start()

In [9]:
df_CPU = stringDF.withColumn('node', regexp_extract('value', r'node_name=(.*?),', 1))\
        .withColumn('source', regexp_extract('value', r'source=(.*?),', 1))\
        .withColumn('process_name', regexp_extract('value', r'process_name=(.*?),', 1))\
        .withColumn('cpu_1min', regexp_extract('value', r'Sub2 total_cpu_one_minute=([^\Wi]+)', 1))\
        .withColumn('cpu_5min', regexp_extract('value', r'total_cpu_five_minute=([^\Wi]+)', 1))\
        .withColumn('cpu_15min',regexp_extract('value', r'total_cpu_fifteen_minute=([^\Wi]+)', 1))\
        .withColumn('process_id',regexp_extract('value', r'process_id=([^\Wi]+)', 1))\
        .withColumn('process_cpu_1min',regexp_extract('value', r'process_cpu_one_minute=([^\Wi]+)', 1))\
        .withColumn('process_cpu_5min',regexp_extract('value', r'process_cpu_five_minute=([^\Wi]+)', 1))\
        .withColumn('process_cpu_15min',regexp_extract('value', r'process_cpu_fifteen_minute=(.*?)i', 1))

In [10]:
# df_CPU = df_CPU.withColumn('timestamp',regexp_extract('process_cpu_15min', r'i\s(.*?)$', 1))\
#             .withColumn('process_cpu_15min_rm',regexp_extract('process_cpu_15min', r'(.*?)i', 1))\
#             .drop('value')\
#             .drop('process_cpu_15min')

In [11]:
# df_CPU = df_CPU.withColumnRenamed('process_cpu_15min_rm','process_cpu_15min')
df_CPU = df_CPU.drop('value')
df_CPU = df_CPU.filter(df_CPU.node!='')

In [12]:
# df_generic_counter.writeStream.format('console').start()
# df_CPU.writeStream.format('console').option("numRows",60).start()

In [13]:
df_CPU.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("path", "s3a://stream-cpu-elab9070/data") \
    .option("checkpointLocation", "s3a://stream-cpu-elab9070/checkpoint") \
    .start()
#     .toTable("Table_cpu_Elab9070")

# df_CPU.write.option("path","s3a://stream-cpu-elab9070/mytable").saveAsTable("default.testcpu")

23/10/22 21:03:37 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/10/22 21:03:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [14]:
df_generic_counter.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("path", "s3a://stream-interface-elab9070/data") \
    .option("checkpointLocation", "s3a://stream-interface-elab9070/checkpoint") \
    .start()
# #     .toTable("Table_interface_Elab9070")

23/10/22 21:03:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [15]:
spark.streams.awaitAnyTermination

<bound method StreamingQueryManager.awaitAnyTermination of <pyspark.sql.streaming.StreamingQueryManager object at 0x7f5ec6f82340>>

23/10/22 21:04:46 WARN FileStreamSinkLog: Compacting took 2498 ms for compact batch 23159


23/10/22 21:29:56 WARN FileStreamSinkLog: Compacting took 2032 ms for compact batch 23189


23/10/23 01:16:08 WARN FileStreamSinkLog: Compacting took 2026 ms for compact batch 23339


23/10/23 08:17:52 WARN FileStreamSinkLog: Compacting took 2026 ms for compact batch 23849


...

23/10/23 19:57:48 WARN FileStreamSinkLog: Compacting took 2018 ms for compact batch 24869


23/10/23 20:18:48 WARN FileStreamSinkLog: Compacting took 2040 ms for compact batch 24769


23/10/23 20:55:49 WARN FileStreamSinkLog: Compacting took 2021 ms for compact batch 24939


23/10/23 21:10:49 WARN FileStreamSinkLog: Compacting took 2129 ms for compact batch 24959


23/10/23 21:46:59 WARN FileStreamSinkLog: Compacting took 2039 ms for compact batch 25009
